In [1]:
import numpy as np
import networkx as nx  # Grafos
import matplotlib.pyplot as plt
import pandas as pd
from time import time
from math import ceil
from itertools import islice
from gurobipy import *

In [2]:
def k_shortest_paths(G, source, target, k, weight=None):
    return list(
        islice(nx.shortest_simple_paths(G, source, target, weight=weight), k)
    )

In [3]:
def generate_instance(n, M, p, prob_1, prob_2, print_r=False):
    """
    param n: Número de nodos + 1
    param M: Número de vehículos
    param p: Número de productos
    param prob_1: Probabilidad de NO conectar dos nodos
    param prob_2: Probabilidad de NO conectar O y D con los nodos
    param print_r: Controla si se hacen o no algunas impresiones
    """
    if print_r:
        print("Lista :",[n,M,p, prob_1, prob_2])
    np.random.seed(2)
    V = tuplelist(range(1,n)) # Nodos
    V_O = ["O"]+[i for i in range(1,n)] # Salida
    V_D = [i for i in range(1,n)]+["D"] # Destino
    #Cross-docks
    V1 = ("O","D") # Solo O y D
    #Arcos y sus costos
    L=[] #Arcos
    for i in V :
        for j in V:
            if np.random.random() > prob_1:
                L.append((i,j))
    if print_r:
        print("L:",L)
    a1 = {(i,j): np.random.randint(80,150) for (i,j) in L if (i!=j and i<j)}
    a2 = {(j,i): a1[i,j]  for (i,j) in L if (i!=j and i<j)}
    a = {**a1, **a2}
    # b
    L1=[]
    for i in V1 :
        for j in V:
            if np.random.random() > prob_2:
                L1.append((i,j))
    if print_r:
        print("L1:",L1)
    b1 = {(i,j): np.random.randint(80,150) for (i,j) in L1 if i =="O" }
    b2 = {(j,i): np.random.randint(80,100) for (i,j) in L1 if i =="D" }
    A,c= multidict({**a, **b1, **b2})
    if print_r:
        print("A:",A)
    if print_r:
        print("C:",c)
    # Costos Operativos
    tc={v+1: np.random.randint(250,500) for v in range(M) }
    if print_r:
        print("tc:",tc)
    #Oferta
    O = {(i, r+1) : np.random.randint(65,100) for i in V for r in range(p) }
    if print_r:
        print("Oferta:",O)
    #Demanda
    Dem = {r+1: np.random.randint(250,300) for r in range(p) }
    if print_r:
        print("Demanda",Dem)
    if print_r:
        print("Aristas:", len(A))
    return V, A, c, tc, O, Dem

In [4]:
def generate_initial(Q, V, A, C, tc, O, Dem,n, M, p, plot_g=False, repeat_end=False, print_=False, perc=False):
    t = time()
    V_O = ["O"] + V # Salida
    V_D = V + ["D"] # Destino
    #Cross-docks
    V1 = ["O","D"] # Solo O y D

    V_o = {} #disponible en vertices
    for i, j in O:
        if i in V_o:
            V_o[i][j] = O[(i, j)]
        else:
            V_o[i] = {}
            V_o[i][j] = O[(i, j)]
    o_nodos = {i: V_o[i].copy() for i in V_o}  # Oferta en otro formato
    o_nodos['D'] = {i+1: 0 for i in range(p)}

    carga_v = {i: {} for i in tc.keys()}  # Carga de cada vehículo
    visited_v = {i: [] for i in tc.keys()}  # Camino de cada vehículo

    dem_c = Dem.copy()
    cost_v = dict(sorted(tc.items(), key=lambda item: item[1]))  # Vehiculos por costo
    charged_h = {i: {j: {k+1: 0 for k in range(p)} for j in V} for i in cost_v}
    usar = ceil(sum(Dem.values())/Q)
    sorted_v = list(cost_v.keys())
    # sorted_v = sorted_v[:usar]
    # k_paths = max(usar ** 3, 2 * len(V))
    print('Use: ', usar)
    print('Perce: ', sum(Dem.values())/usar)
    if perc:
        Q = min(sum(Dem.values())/usar, Q)
        Q = ceil(Q)

    def restante(i):
        return Q - sum(carga_v[i].values())

    def cargar(i, prod, value, nodo):
        if nodo == 'D':
            return
        if prod in carga_v[i]:
            carga_v[i][prod] += value
        else:
            carga_v[i][prod] = value
        charged_h[i][nodo][prod] += value

    def get_p(i, prod, value):
        v = o_nodos[i][prod]
        if v < value:
            raise ValueError('No hay oferta suficiente')
        o_nodos[i][prod] -= value

    def get_d(dem, prod, value):
        dem[prod] -= value

    def oferta_d(nodo, demanda):
        total = 0
        for i in demanda:
            if i in o_nodos[nodo]:
                aux = o_nodos[nodo][i]
                total += min(aux, demanda[i])
        return total

    def get_o(nodo, demanda, capacidad):
        oferta = {}
        for i in demanda:
            if i in o_nodos[nodo]:
                aux = o_nodos[nodo][i]
                aux = min(aux, demanda[i], capacidad)
                capacidad -= aux
                oferta[i] = aux
                if capacidad == 0:
                    break
        return oferta


    def generate_graph(arcos, plot=True):
        G = nx.DiGraph()
        for i in arcos:
            G.add_edge(*i, weight=arcos[i])
        if plot:
            pos = nx.circular_layout(G)
            nx.draw(G, pos, with_labels=True)
            plt.show()
        return G

    graph = generate_graph(C, plot_g)

    # all_paths = k_shortest_paths(graph, 'O', 'D', k_paths, weight='weight')
    all_paths = nx.shortest_simple_paths(graph, 'O', 'D', weight='weight')
    # print('Paths: ', all_paths)

    def oferta_path(path):
        oferta = {}
        for i in path:
            if i in o_nodos:
                for j in o_nodos[i]:
                    if j in oferta:
                        oferta[j] += o_nodos[i][j]
                    else:
                        oferta[j] = o_nodos[i][j]
        return oferta


    def arcos_use_list(nodo, arcos, visit):
        arcos_use = []
        for i, j in arcos:
            if i == nodo and j not in visit[v]:
                arcos_use.append((i, j))
        return {i: arcos[i] for i in arcos_use}
    
    
    global costo
    costo = 0
    
    def try_take_demand(path_list, v):
        global costo
        for path in path_list: # recorrer el camino en el conjunto de caminos que calculé antes
            oferta_this_path = oferta_path(path) # obtengo la oferta de ese camino
            # print('Path: ', path, 'Dem: ', dem_c, 'Tot: ', sum(oferta_this_path.values()))
            aux = pd.DataFrame({'Demanda': dem_c, 'Oferta': oferta_this_path})
            if sum(aux.min(axis=1)) < Q: #si la demanda es menor de la capacidad del vehículo, no me interesa llenar la capacidad sino satisfacer la demanda 
                # print('use demand')
                aux = aux['Demanda'] > aux['Oferta'] # si la demanda es mayor que la oferta
                if aux.sum() == 0: # si en ningun camino la demanda es mayor que la oferta, entonces mi oferta es suficiente para satisfacer la demanda
                    visited_v[v] = path # entonces este camino me sirve y lo agrego 
                else:
                    continue # si no me voy a otro camino
            else:
                # print('Use cap, Q: ', Q, 'off: ', sum(oferta_this_path.values()))
                if Q <= sum(oferta_this_path.values()): #la capacidad debe ser menor o igual que lo que me da de oferta ese camino
                    visited_v[v] = path # este es el camino indicado
                else:
                    continue # sino sigo buscando el camino
            for new_nodo in path[::-1]:
                if new_nodo in ['O', 'D']:
                    continue
                capacidad = restante(v) # capacidad es el restante del vehículo
                ofert = get_o(new_nodo, dem_c, capacidad) # revisa la oferta de ese nodo en ese momento segun tambien la demanda y capacidad
                for i in ofert: # de esta oferta 
                    get_p(new_nodo, i, ofert[i]) # voy obteniendo el valor de ese producto
                    cargar(v, i, ofert[i], new_nodo) # carago el vehículo
                    get_d(dem_c, i, ofert[i]) # y reduzco la demanda
            costo += nx.path_weight(graph, path, weight="weight") #cuando ya elijo el camino y le sumo todo lo que cuesta ese camino
            return True
        return False

    
    used_paths = []
    for v in sorted_v:
        # print('Dem: ', dem_c)
        if sum(dem_c.values()) == 0:
            break
        while not try_take_demand(used_paths, v): #mientras no haya un camino corto de O a D que cumpla la condición de que la oferta total del camino es suficiente para llenar el vehículo o satisfacer la demanda general
            used_paths = used_paths + list(islice(all_paths, 0, 1)) #buscar el siguiente camino más corto de 'O' a 'D' y añadirlo a used_paths
    print('Used paths: ', used_paths)
    costo_op = 0
    Tot_c = {}
    for i in charged_h:
        if len(visited_v[i]) > 0:
            costo_op += tc[i]
            print('El vehículo', i, 'cargó: ')
            df = pd.DataFrame(charged_h[i]).T
            df = df.loc[df.sum(axis=1) > 0, :].copy()
            df.index.name = 'Nodo'
            df.columns = ['Producto ' + str(i) for i in df.columns]
            print(df)
            print('Camino: ', visited_v[i])
            Tot_c[i] = sum(carga_v[i].values())
            print('Total: ', Tot_c[i])
            print('\n\n')
        else:
            Tot_c[i] = 0

    print('Costo del recorrido: ', costo)           
    print('Costo operativo: ', costo_op)
    print('Costo total: ', costo_op + costo)
    edges_usable = set()
    for i in used_paths:
        for j in range(1, len(i)):
            edges_usable.add((i[j - 1], i[j]))
    print(edges_usable)
    X = {}
    Y = {}
    A_s = {}
    U = {}
    for v in visited_v:
        for i, j in A:
            X[i, j, v] = 0
        for i in V:
            Y[i, v] = 0
            U[i, v] = n
            for j in Dem:
                A_s[i, j, v] = 0
        path = visited_v[v]
        pos = n
        print('Auto: ', v, 'Path: ', path)
        for i in range(1, len(path)):
            X[path[i-1], path[i], v] = 1
        aux = path.copy()
        aux.reverse()
        for i in aux:
            if i in ['O', 'D']:
                continue
            U[i, v] = pos
            pos -= 1
            Y[i, v] = 1
    #             if i == aux[-2]:
    #                 print('Set 1: i=',i,'aux=',aux)
    #                 U[i, v] = 1

    A_s = {(i, r+1, v+1): 0 for i in V for r in range(p) for v in range(M)}
    # i vehiculo
    for v in charged_h:
        for i in charged_h[v]:
                for p in charged_h[v][i]:
                    A_s[i, p, v] = charged_h[v][i][p]
    t = time() -  t
    print('Time: ', t)
    return X, Y, A_s, U, costo_op + costo, edges_usable

In [5]:
#grafo NO COMPLETO
def auto(S, use_start=True,print_=True, perc=False):
    """
    Para cada elemento de S genera una instancia y la resuelve.
    
    param S: Lista de instancias
    param use_start: Si es verdadero utiliza valores de inicio por heurística, caso contrario no.
    param print_: Si es verdadero imprime varias variables generadas y una imagen del grafo creado.
    """
    result = {}
    idx_i = 1
    for i in S:
        t_s_all = time()
        result[idx_i] = {}
        try:
            [n,M,p,l1,l2]=i #n: Nodos, M: Vehículos, p: Productos
        except ValueError:
            [n,M,p]=i #n: Nodos, M: Vehículos, p: Productos
            l1 = l2 = 0.5
        # si no se ingresan las probabilidades se usa 0.5
        V, A, c, tc, O, Dem = generate_instance(n, M, p, l1, l2, print_)
        result[idx_i]['Instancia'] = str([n-1,len(A),M,p])
        #Capacidades del vehículo por producto
        Q = 200
        start_r = time()
        X0, Y0, A0, U0, costo_in, n_A = generate_initial(Q, V, A, c, tc, O, Dem, n, M, p, plot_g=print_, repeat_end=False,perc=perc)
        end_r = time()
        V_O = ["O"]+[i for i in range(1,n)] # Salida
        V_D = [i for i in range(1,n)]+["D"] # Destino
        #Cross-docks
        V1 = tuplelist(["O","D"]) # Solo O y D
        
        m = Model("Problema-Tesis")

        # Crear las variables
        # X: Voy de i a j para vehiculo v: [1,0]
        x = m.addVars([(i, j, v+1) for (i,j) in A for v in range(M)], name="x", vtype=GRB.BINARY)
        # Y: Vehiculo pasa por i
        y = m.addVars([(i, v+1) for i in V for v in range(M) ], name="y", vtype=GRB.BINARY)
        # A: Peso de productor r en nodo i por v
        a = m.addVars([(i, r+1, v+1) for i in V for r in range(p) for v in range(M)], name = "a", vtype = GRB.INTEGER)
        # Crear las variables de ordenamiento de nodos
        u = m.addVars([(i, v+1) for i in V for v in range(M)], name="u", vtype=GRB.INTEGER, lb=1, ub=n) #u_i \in {1,2,...,n}
        
        #Función Objetivo
        
        C_V = {(i,j,v+1): c[i,j]  for (i,j) in A for v in range(M)}
        TC =  {("O",j,v+1): tc[v+1] for j in V for v in range(M)}
        
        m.setObjective(x.prod(C_V) + x.prod(TC, "O","*","*") , GRB.MINIMIZE)

        #Restricciones
        print('\n\nRestricciones\n\n')
        m.addConstrs(( x.sum("O","*",v+1) <=1  for v in range(M)))
        for v in range(M):
            s = 0
            for i, j, k in x:
                if i!='O':
                    continue
                if k == v + 1:
                    s += X0[i, j, k]
            if s > 1:
                print('Error sum_XO: v=',v+1)
        m.addConstrs(( x.sum("*","D",v+1) <=1  for v in range(M)))
        for v in range(M):
            s = 0
            for i, j, k in x:
                if j!='D':
                    continue
                if k == v + 1:
                    s += X0[i, j, k]
            if s > 1:
                print('Error sum_XD: v=',v+1)
        m.addConstrs(( x.sum("*",j,v+1) == y[j,v+1] for j in V for v in range(M)))
        for  v in range(M):
            for j in V:
                s = 0
                for i1, j1, k1 in x:
                    if j1 == j and k1 == v+1:
                        s += X0[i1, j1, k1]
                if s != Y0[j,v+1]:
                    print('Error sum_x=y: j=',j,'Veh:',v+1)
        m.addConstrs(( x.sum("*",k,v+1)-x.sum(k,"*",v+1)  == 0 for k in range(1,n) for v in range(M)))
        for  v in range(M):
            for k in range(1,n):
                s1 = 0
                s2 = 0
                for i1, j1, k1 in x:
                    if j1 == k and k1 == v+1:
                        s1 += X0[i1, j1, k1]
                    if i1 == k and k1 == v+1:
                        s2 += X0[i1, j1, k1]
                if s1 != s2:
                    print('Error sum_x=sum_x: k=',k,'Veh:',v+1)

        #variables auxiliares de ordenamiento  u_i  para los nodos i in V \{O},
        #que indican la posición de cada nodo dentro del tour, asumiendo que el nodo O ocupa la primera posición.
        #se eliminan soluciones con subtoures, empleando m−2n+2  restricciones, donde m=n(n−1) es el número de arcos en el grafo. Que no requiere un número exponencial de restricciones

        # para cualquier arco (i,j) cuyos dos extremos sean distintos al nodo  O , si el arco es seleccionado dentro de la solución, 
        # entonces debe cumplirse que uj≥ui+1
        # Restricciones de ordenamiento de nodos
        for v in range(M):
            m.addConstrs((u[j, v+1] >= u[i,v+1] + (1 + n)*x[i,j,v+1]-n  for i,j in A if i!="O" and j!="D"))
            for i, j in A:
                if i!="O" and j!="D":
                    if U0[j, v+1] >= U0[i,v+1] + (1 + n)*X0[i,j,v+1]-n:
                        continue
                    # if U0[j, v+1] < U0[i,v+1] + (1 + n)*X0[i,j,v+1]-n:
                    print('Error: ')
                    print('Uj: ', U0[j, v+1], 'Debe: >=')
                    print('Ui: ', U0[i, v+1])
                    print('X: ', X0[i,j,v+1]) # 3 a 8
                    print('i: ', i, 'j: ', j, 'v: ', v+1)
        
        m.addConstr(( x.sum("O","*","*") <= M))
        s = 0
        for i1, j1, k1 in x:
            if i1 != 'O':
                continue
            s += X0[i1, j1, k1]
        if s > M:
            print('Error origin sum')
        m.addConstrs((x[i,j,v] + x[j,i,v] <=1 for v in range(1,M+1) for i,j in A if i!=j and i!="O" and j!="D"))
        for v in range(1,M+1):
            for i,j in A:
                if i!=j and i!="O" and j!="D":
                    if X0[i,j,v] + X0[j,i,v] > 1:
                        print('Error return: i=',i,'j=',j)
        m.addConstrs((a.sum(i, r+1, "*") <= O[i, r+1] for i in V for r in range(p)))
        for i in V:
            for r in range(p):
                s = 0
                for i1, j1, k1 in a:
                    if i1 == i and j1 == r+1:
                        s += A0[i1, j1, k1]
                if s > O[i, r+1]:
                    print('Error oferta: i=',i,'prod=',r+1,'sum=',s)
        m.addConstrs((a.sum("*","*",v+1) <= Q for v in range(M)))
        for v in range(M):
            s = 0
            for i1, j1, k1 in a:
                if k1 == v+1:
                    s += A0[i1, j1, k1]
            if s > Q:
                print('Error capacidad: v=',v+1)
        m.addConstrs((a[i, r+1, v+1]<= O[i,r+1]*y[i, v+1] for i in V for r in range(p) for v in range(M)))
        for i in V:
            for r in range(p):
                for v in range(M):
                    if A0[i, r+1, v+1] > O[i,r+1]*Y0[i, v+1]:
                        print('Error oferta ind: i=',i,'prod=',r+1,'veh=', v+1)
        m.addConstrs((a.sum("*", r+1, "*") == Dem[r+1] for r in range(p)))
        for r in range(p):
            s = 0
            for i1, j1, k1 in a:
                if j1 == r+1:
                    s += A0[i1, j1, k1]
            if s != Dem[r+1]:
                print('Falta demanda: r=',r+1,'s=',s)
        # Terminar luego de 3600 segundos
        m.Params.TimeLimit = 3600
        result[idx_i]['Sart'] = costo_in
        result[idx_i]['Creacion & heurística'] = end_r - start_r
        if use_start:
            result[idx_i]['Usa inicio'] = 'Sí'
            for i, j, k in X0:
                x[i, j, k].setAttr('Start', X0[(i, j, k)])
            for i, j in Y0:
                y[i, j].setAttr('Start', Y0[(i, j)])
            for i, j, k in A0:
                a[i, j, k].setAttr('Start', A0[(i, j, k)])
            for i, j in U0:
                u[i, k].setAttr('Start', U0[(i, j)])
            for i, j in A:
                if (i, j) not in n_A:
                    for v in range(M):
                        x[i, j, k].setAttr('ub', 0)
        else:
            result[idx_i]['Usa inicio'] = 'No'
        m.update()
        start_r = time()
        m.optimize()
        end_r = time()
        result[idx_i]['Obj. Value'] = m.getObjective().getValue()
        result[idx_i]['Gap'] = m.MIPGap
        result[idx_i]['CPU time (seg)'] = end_r - start_r
        m.write("Pruebas-varias-instancias-completo.lp")
        if m.status == GRB.Status.OPTIMAL or GRB.Status.TIME_LIMIT:
            # Recuperar los valores de las variables
            vx = m.getAttr('x', x)
            print('Arcos seleccionados:')
            dict_route = {}
            for v in range(M):
                dict_route[v + 1] = []
                for (i,j) in A:
                    if i != 'O':
                        continue
                    if vx[i,j,v+1] >= 0.99:
                        dict_route[v + 1].append(i)
                        dict_route[v + 1].append(j)
                        while dict_route[v + 1][-1] != 'D':
                            for k in V + ['D']:
                                if (dict_route[v + 1][-1], k) not in A:
                                    continue
                                if(vx[dict_route[v + 1][-1], k, v+1] >= 0.99):
                                    dict_route[v + 1].append(k)
                                    break
                        break
                if len(dict_route[v + 1]) > 0:
                    print('Auto ', v + 1, ': ', dict_route[v + 1])
        print("---------------")
                
        if m.status == GRB.Status.OPTIMAL or GRB.Status.TIME_LIMIT:
            # Recuperar los valores de las variables
            for v in range(M):
                for i in V:
                    if y[i,v+1].x>= 0.99:
                        print(i,v+1)
        print("---------------")
        if m.status == GRB.Status.OPTIMAL or GRB.Status.TIME_LIMIT:
            for v in range(M):
                s=0
                for r in range(p):
                    for i in V:
                        if a[i,r+1, v+1].x>0:
                            s=s+a[i, r+1, v+1].x
                print("El vehículo {} recogió en total un peso de {}".format(v+1,s))               
        print("------")
        if m.status == GRB.Status.OPTIMAL or GRB.Status.TIME_LIMIT:
            #Recuperar el valor de las variables
            for v in range(M):
                for r in range(p):
                    for i in V:
                        if a[i,r+1, v+1].x>0:
                            print('Cantidad: {} recogida en el nodo{} de producto{} por el auto{} '.format(a[i, r+1, v+1].x,i,r+1,v+1))                               

#         for v in range(M):
#             print('Auto: ', v+1,': ',end=' ')
#             for i, j in u:
#                 if j == v + 1:
#                     print(f'u{i},{j}={u[i,j].x}', end=' ')
#             print(' ')
        print("---------------")
        result[idx_i]['Total time'] = time() - t_s_all
        idx_i += 1
    return result

In [ ]:
S=[[5,4,2,0,0],[5,5,2,0.4,0.4],[5,5,2,0,0],[6,4,2,0.3,0.3],[6,4,2,0,0],[6,5,2,0,0],[6,5,3,0.3,0.3],[6,8,3,0.3,0.3],
   [7,5,3,0.3,0.3],[7,6,3,0.5,0.5],[7,6,3,0.4,0.4],[8,6,3,0.5,0.5],[8,7,3,0.5,0.5],[8,6,3,0.4,0.4],[9,6,3,0.4,0.4],
   [9,6,3,0.5,0.5],[10,7,4,0.6,0.6],[10,9,4,0.52,0.52],[10,7,4,0.5,0.5],[10,9,4,0.5,0.5],[11,7,4,0.6,0.6],[11,8,4,0.6,0.6],
   [11,8,4,0.55,0.55],[12,8,4,0.6,0.6],[12,10,4,0.6,0.6],[13,7,4,0.61,0.61],[16,11,3,0.5,0.5],[19,11,4,0.5,0.5],
   [20,12,5,0.5,0.5],[22,13,5,0.5,0.5],[24,14,6,0.5,0.5],[25,15,6,0.5,0.5]]
r = auto(S, True, True)
report = pd.DataFrame(r).T
report.index.name = 'Prueba'
report.to_excel('Inicio+modelo_BIEN_reducir_posibilidades.xlsx')